In [1]:
import networkx as nx
from networkx.readwrite import json_graph
import community as community

import pandas as pd
import numpy as np

import itertools
import math
import csv
import json
from operator import itemgetter

import matplotlib.pyplot as plt
import mplleaflet
from pylab import show
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<h1> Get the node and edge (link) data and load into the network

In [2]:
#Get the station data 
ndata = pd.read_csv("data/nyc_subway_stations_with_lines.csv")

In [3]:
ndata.head(3)

,Unnamed: 0,stop_id,stop_name,stop_lat,stop_lon,Borough,Daytime Routes
0,0,101,Van Cortlandt Park - 242 St,40.889248,-73.898583,Bx,1
1,1,103,238 St,40.884667,-73.900870,Bx,1
2,2,104,231 St,40.878856,-73.904834,Bx,1


In [4]:
#initialize the graph
#G.clear()
G = nx.MultiDiGraph(name='NYC Subway Network')

In [5]:
#add nodes and attributes
for i in range(0,len(ndata)):
    G.add_node(ndata.stop_id[i], name = ndata.stop_name[i], line = ndata['Daytime Routes'][i], boro = ndata.Borough[i], lat = ndata.stop_lat[i], lng = ndata.stop_lon[i])

In [6]:
#show the nodes in G
#G.nodes(data=True)

In [7]:
#get the edge data and attributes
edata = pd.read_csv("data/subway_duration_between_stops_weekday(common_route).csv")

In [8]:
edata.head(2)

,Unnamed: 0,bound,duration,from_stop,from_stop_id,from_stop_sequence,route_id,service_id,to_stop,to_stop_id,to_stop_sequence,train
0,0,N,180,Prospect Park,D26N,1,A..N55R,B20170625WKD,Botanic Garden,S04N,2,FS
1,1,N,120,Botanic Garden,S04N,2,A..N55R,B20170625WKD,Park Pl,S03N,3,FS


In [9]:
#add node id to edge data
edata['source_node']= edata['from_stop_id'].str[:3]
edata['target_node']= edata['to_stop_id'].str[:3]

In [10]:
edata.head(2)

,Unnamed: 0,bound,duration,from_stop,from_stop_id,from_stop_sequence,route_id,service_id,to_stop,to_stop_id,to_stop_sequence,train,source_node,target_node
0,0,N,180,Prospect Park,D26N,1,A..N55R,B20170625WKD,Botanic Garden,S04N,2,FS,D26,S04
1,1,N,120,Botanic Garden,S04N,2,A..N55R,B20170625WKD,Park Pl,S03N,3,FS,S04,S03


In [11]:
#ADD THE EDGES TO THE NETWORK
for i in range(0,len(edata)):
    G.add_edge(edata.source_node[i],edata.target_node[i], 
               weight = edata.duration[i],
               duration = edata.duration[i],
               bound = edata.bound[i],
               demand = 1,
               platform = edata.train[i])

In [12]:
#ADD TRANSFER WITHOUT SWIPE EDGES
tdata = pd.read_csv("data/transfer_without_swipe.csv")
tdata['source_node']= tdata['from_stop_id']
tdata['target_node']= tdata['to_stop_id']
tdata.head(3)

,Unnamed: 0,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line,source_node,target_node
0,0,112,A09,2,180,168 St - Washington Hts,168 St,1,A,112,A09
1,1,112,A09,2,180,168 St - Washington Hts,168 St,1,C,112,A09
2,2,120,120,2,180,96 St,96 St,1,2,120,120


In [13]:
for i in range(0,len(tdata)):
    G.add_edge(tdata.source_node[i],tdata.target_node[i],
               weight = tdata.min_transfer_time[i],
               duration = tdata.min_transfer_time[i],
               bound = 'T',
               demand = 1,
               platform = tdata.from_line[i]+" "+ tdata.to_line[i])

<h1> Take a look at the network, summarize characteristics

In [ ]:
#print network info
print nx.info(G)

In [14]:
#graph density for directed graphs isd = \frac{m}{n(n-1)}, where n is the number of nodes and m is the number of edges
#The density is 0 for a graph without edges and 1 for a complete graph. The density of multigraphs can be higher than 1.
#Self loops are counted in the total number of edges so graphs with self loops can have density higher than 1.
nx.density(G)

0.010294699926466429

In [ ]:
#plot network
plt.axis("off")
spring_pos = nx.spring_layout(G)
circular =nx.circular_layout(G)
random = nx.random_layout(G)
nx.draw_networkx(G, pos = spring_pos, with_labels = False, alpha=0.3,node_size = 35)

In [ ]:
#Method 1 - betweenness_centrality
#compute the betweeness centrality to determine who is the most "important" individual in the network.
#using betweenness centrality is a measure of how many shortest paths pass through a particular vertex.
#Method from http://glowingpython.blogspot.com/2013/02/betweenness-centrality.html
#we need to convert our network to an undirected one first
G_u = G.to_undirected()

def most_important(G):

  ranking = nx.betweenness_centrality(G).items()
  
  r = [x[1] for x in ranking]
  m = sum(r)/len(r) # mean centrality
  t = m*3 # threshold, we keep only the nodes with 3 times the mean
  Gt = G.copy()
  for k, v in ranking:
    if v < t:
      Gt.remove_node(k)
  return Gt



In [ ]:
dominant = most_important(G_u)

In [ ]:
#print dominant nodes and its neighbors
#for i in range(1,len(dominant.nodes())):
    #print (dominant.neighbors(dominant.nodes()[i]));
    
#print dominant nodes and node attributes
dominant.nodes(data=True)

In [ ]:
#Plot betweeness centrality dominant nodes
plt.axis("off")

# draw the nodes and the edges (all)
nx.draw_networkx_nodes(G,spring_pos,node_color='b',alpha=0.2,node_size=10)
nx.draw_networkx_edges(G,spring_pos,alpha=0.1)

# draw the most important nodes with a different style
nx.draw_networkx_nodes(dominant,spring_pos,node_color='r',alpha=0.4,node_size=100)
# also the labels this time
#nx.draw_networkx_labels(dominant,spring_pos,font_size=10,font_color='b')
show()

In [ ]:
#determine most important node by spectral theory power method
def most_important(G):

  ranking = nx.eigenvector_centrality_numpy(G).items()
  
  r = [x[1] for x in ranking]
  m = sum(r)/len(r) # mean centrality
  t = m*3 # threshold, we keep only the nodes with 3 times the mean
  Gt = G.copy()
  for k, v in ranking:
    if v < t:
      Gt.remove_node(k)
  return Gt

dominante3 = most_important(G_u)

In [ ]:
#print spectral dominant nodes and its neighbors
#for i in range(1,len(dominante3.nodes())):
    #print (dominante3.neighbors(dominante3.nodes()[i]));

#print dominant nodes and node attributes
dominante3.nodes(data=True)

In [ ]:
#Plot spectral dominant nodes
plt.axis("off")

# draw the nodes and the edges (all)
nx.draw_networkx_nodes(G,spring_pos,node_color='b',alpha=0.2,node_size=10)
nx.draw_networkx_edges(G,spring_pos,alpha=0.1)

# draw the most important nodes with a different style
nx.draw_networkx_nodes(dominante3,spring_pos,node_color='r',alpha=0.4,node_size=100)
# also the labels this time
#nx.draw_networkx_labels(dominante3,spring_pos,font_size=10,font_color='b')
show()

In [ ]:
#plotting on a map
ndatamap = ndata[['stop_id','stop_lon','stop_lat']]
pos=ndatamap.set_index('stop_id').T.to_dict('list')

fig, ax = plt.subplots(figsize=(15,15))

nx.draw_networkx_nodes(G,pos=pos,node_size=10,node_color='red',edge_color='k',alpha=.5, with_labels=True)
nx.draw_networkx_edges(G,pos=pos,edge_color='gray', alpha=.1)
nx.draw_networkx_labels(G,pos, label_pos =10.3)


mplleaflet.display(fig=ax.figure)


<h1>FIND SHORTEST PATHS AND CALCULATE NETWORK METRIC

In [15]:
#Dijkstra’s algorithm to find the shortest path in a weighted network
short_paths = nx.all_pairs_dijkstra_path(G, weight = 'weight')
sp = pd.DataFrame(short_paths)
sp.to_csv('data/shortest_paths')
sp.head(2)

,101,103,104,106,107,108,109,110,111,112,...,S22,S23,S24,S25,S26,S27,S28,S29,S30,S31
101,[101],"[103, 101]","[104, 103, 101]","[106, 104, 103, 101]","[107, 106, 104, 103, 101]","[108, 107, 106, 104, 103, 101]","[109, 108, 107, 106, 104, 103, 101]","[110, 109, 108, 107, 106, 104, 103, 101]","[111, 110, 109, 108, 107, 106, 104, 103, 101]","[112, 111, 110, 109, 108, 107, 106, 104, 103, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,"[101, 103]",[103],"[104, 103]","[106, 104, 103]","[107, 106, 104, 103]","[108, 107, 106, 104, 103]","[109, 108, 107, 106, 104, 103]","[110, 109, 108, 107, 106, 104, 103]","[111, 110, 109, 108, 107, 106, 104, 103]","[112, 111, 110, 109, 108, 107, 106, 104, 103]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Calculate length (duration total) for shortest paths
short_paths_lng = nx.all_pairs_dijkstra_path_length(G, weight = 'weight')
sp_lng = pd.DataFrame(short_paths_lng)
sp_lng.head(2)

,101,103,104,106,107,108,109,110,111,112,...,S22,S23,S24,S25,S26,S27,S28,S29,S30,S31
101,0.0,90.0,180.0,270.0,360.0,450.0,540.0,600.0,690.0,810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,90.0,0.0,90.0,180.0,270.0,360.0,450.0,510.0,600.0,720.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#create combined dataframe for ease of use. this takes over 1 hr to run
nodes = G.nodes()
original_list = pd.DataFrame(columns=('source','target','duration','path'))
for i in G.nodes():
    for p in nodes:
        duration =  sp_lng.at[i,p]
        path = sp.at[i,p]
        row = pd.DataFrame.from_items([('source',[i]),('target',[p]),('duration',duration),('path',[path])])
        original_list = pd.concat([original_list,row])


In [ ]:
#alternate merges - concats one df above the other with double keys 
pieces = {'duration': sp, 'path': sp_lng}
original_listcon = pd.concat(pieces)

In [ ]:
#alternate merges
drow=[]
prow=[]

for d in sp_lng.itertuples():
    drow = drow.append(d.T)
for p in sp.iterrows():
    prow = prow.append(p.transform())
cols = list(sp.columns.values)
rws = list(sp.index.values)
add = [('source',cols),('target',rws),('duration',drow),('path',prow)]
original_list = pd.DataFrame.from_items(add)

In [18]:
#for future use so i don't have to run above
original_list= pd.read_csv("data/sp_original_data.csv")

In [19]:
#reindex 
original_list = original_list.reset_index()

In [20]:
original_list.head(2)

,index,Unnamed: 0,source,target,duration,path
0,0,0,G26,G26,0.0,['G26']
1,1,0,G26,G24,180.0,"['G24', 'G26']"


In [21]:
#total duration for network
orig_sum = original_list.duration.sum(skipna=True)
orig_sum

474117060.0

In [ ]:
#Modify graph to prevent further change by adding or removing nodes or edges.
#nx.freeze(G)

In [22]:
#remove node, recalculate paths and length , save in dict. This takes about 1:25min to run.
duration = {}
paths = {}
for n in G:
    #make a copy of the network
    w_G = G.copy()
    #remove a node from the copy
    w_G.remove_node(n)
    #recaclulate all pairs
    duration["missing_duration_"+n] = nx.all_pairs_dijkstra_path_length(w_G, weight = 'weight')
    paths["missing_paths_"+n] = nx.all_pairs_dijkstra_path(w_G, weight = 'weight') 

In [ ]:
#paths
paths_df = pd.DataFrame.from_dict(paths)
#paths = pd.DataFrame.from_dict(paths)
paths_df.to_csv('data/missing_paths.csv')

In [24]:
duration_df = pd.DataFrame.from_dict(duration)
#paths = pd.DataFrame.from_dict(paths)
duration_df.to_csv('data/missing_duration.csv')

In [ ]:
#remove node pairs, recalculate paths and length , save in dict
duration_pair = {}
paths_pair = {}
for index, row in original_list.iterrows():
    #make a copy of the network
    p_G = G.copy()
    #remove a pair of nodes from the copy
    p_G.remove_node(row['source'])
    if p_G.has_node(row['target']):
        p_G.remove_node(row['target'])
    #recaclulate all pairs
    duration_pair["pair_missing_duration_"+n] = nx.all_pairs_dijkstra_path_length(p_G, weight = 'weight')
    paths_pair["pair_missing_paths_"+n] = nx.all_pairs_dijkstra_path(p_G, weight = 'weight')

    

<h1> Exporting Data

In [ ]:
#to file
paths_pair_df = pd.DataFrame.from_dict(paths_pair)
paths_pair_df.to_csv('data/missing_pair_paths.csv')
duration_pair_df = pd.DataFrame.from_dict(duration_pair)
duration_pair_df.to_csv('data/missing_pair_duration.csv')

In [ ]:
#For future use:

#node data and attributes
node_data = G.nodes(data=True)
nd_data = pd.DataFrame(node_data)
nd_data.to_csv('data/node_data.csv')


#edge data and attributes
edge_data = G.edges(data=True)
eg_data = pd.DataFrame(edge_data)
eg_data.to_csv('data/edge_data.csv')


#get the weights of all the edges (node pairs)
edge_weights = pd.DataFrame(G.edges(data='weight'))
edge_weights.to_csv('data/edge_data.csv')


#Find orphans: nodes with no edges
no_edges = nx.isolates(G)
noedges = pd.DataFrame(no_edges)
noedges.to_csv('data/no_edges.csv')


#export node-link info as json for d3 mapping
data = json_graph.node_link_data(G)
with open('nodelinkdata.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
#send list of original durations and paths to file
original_metric.to_csv('data/original_metric_data.csv')

#send dict of single node removal durations and paths to file
#single_removal_metric.to_csv('data/single_removal_metric_data.csv')

paths = pd.DataFrame(paths)
#paths = pd.DataFrame.from_dict(paths)
paths.to_csv('data/missing_paths.csv')
duartion = pd.DataFrame(duration)
#paths = pd.DataFrame.from_dict(paths)
duration.to_csv('data/missing_duration.csv')


#send dict of single node removal durations and paths to file
pair_removal_metric.to_csv('data/pair_removal_metric_data.csv')